<h2>Titter sentiment analysis</h2>

- You can also multiply score by the number of likes a particular tweet has received

In [65]:
import os
import nltk
import tweepy
from dotenv import load_dotenv
import json
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [58]:
import ssl
def set_up_ssl():
    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        pass
    else:
        ssl._create_default_https_context = _create_unverified_https_context

In [59]:
set_up_ssl()
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/dennisfenchenko/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [22]:
load_dotenv()
TWITTER_BEARER_TOKEN = os.getenv('TWITTER_BEARER_TOKEN')
TWITTER_API_KEY = os.getenv('TWITTER_API_KEY')
TWITTER_API_SECRET_KEY = os.getenv('TWITTER_API_SECRET_KEY')
TWITTER_ACCESS_TOKEN = os.getenv('TWITTER_ACCESS_TOKEN')
TWITTER_ACCESS_TOKEN_SECRET = os.getenv('TWITTER_ACCESS_TOKEN_SECRET')

In [23]:
print("Authenticating to Twitter...")

client = tweepy.Client(bearer_token=TWITTER_BEARER_TOKEN)
auth = tweepy.OAuthHandler(TWITTER_API_KEY, TWITTER_API_SECRET_KEY)
auth.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)

Authenticating to Twitter...


In [24]:
# create Twitter API client
api = tweepy.API(auth, wait_on_rate_limit=True)

In [25]:
democratic_candidate_names = ["@JohnFetterman", "John Fetterman"]
republic_candidate_names = ["@DrOz", "@droz"]

In [86]:
# TEST CODE

# fetching and displaying sample tweets

QUERY = "John Fetterman OR @JohnFetterman -filter:retweets"
SINCE_DATE = "2022-10-03"
UNTIL_DATE = "2022-10-10"

tweets = api.search_tweets(q=QUERY, count = 2, tweet_mode="extended",
                           lang="en", result_type="popular", since_id =
                           SINCE_DATE, until = UNTIL_DATE)

In [87]:
# TEST CODE

tweets

[Status(_api=<tweepy.api.API object at 0x1089248b0>, _json={'created_at': 'Wed Oct 05 17:39:10 +0000 2022', 'id': 1577715031621439494, 'id_str': '1577715031621439494', 'full_text': "John Fetterman doesn't live in New Jersey, won't vote in Turkey's elections, and never killed puppies.\n\nHis opponent can't say that.", 'truncated': False, 'display_text_range': [0, 132], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'metadata': {'result_type': 'popular', 'iso_language_code': 'en'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1640929196, 'id_str': '1640929196', 'name': 'BrooklynDad_Defiant!☮️', 'screen_name': 'mmpadellan', 'location': 'Brooklyn, NY', 'description': "Proud papa. Perpetually pissed.\n#BlueWave2022, #Feminist. \n#IStandWithUkraine🇺

In [82]:
# TEST CODE

tweet_status = tweets[1]
print(tweet_status)
# print("tweets ", len(tweets))

Status(_api=<tweepy.api.API object at 0x1089248b0>, _json={'created_at': 'Sun Oct 09 19:44:21 +0000 2022', 'id': 1579196087578943488, 'id_str': '1579196087578943488', 'full_text': '‘Dr Oz has never stopped reminding me that I had a stroke,’ @JohnFetterman says at top of a rally in Bucks County.  ‘I guarantee there’s at least one person here filing me hoping to catch me missing some words. What kind of a doctor roots for somebody who was sick, stay sick?’', 'truncated': False, 'display_text_range': [0, 277], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'JohnFetterman', 'name': 'John Fetterman', 'id': 3622368202, 'id_str': '3622368202', 'indices': [60, 74]}], 'urls': []}, 'metadata': {'result_type': 'popular', 'iso_language_code': 'en'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': Non

In [51]:
# TEST CODE

json_tweet_str = json.dumps(tweet_status._json)

json_tweet = json.loads(json_tweet_str)

In [52]:
# TEST CODE
json_tweet

{'created_at': 'Sun Oct 09 19:44:21 +0000 2022',
 'id': 1579196087578943488,
 'id_str': '1579196087578943488',
 'full_text': '‘Dr Oz has never stopped reminding me that I had a stroke,’ @JohnFetterman says at top of a rally in Bucks County.  ‘I guarantee there’s at least one person here filing me hoping to catch me missing some words. What kind of a doctor roots for somebody who was sick, stay sick?’',
 'truncated': False,
 'display_text_range': [0, 277],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'JohnFetterman',
    'name': 'John Fetterman',
    'id': 3622368202,
    'id_str': '3622368202',
    'indices': [60, 74]}],
  'urls': []},
 'metadata': {'result_type': 'popular', 'iso_language_code': 'en'},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_

In [53]:
# TEST CODE
json_tweet["full_text"]

'‘Dr Oz has never stopped reminding me that I had a stroke,’ @JohnFetterman says at top of a rally in Bucks County.  ‘I guarantee there’s at least one person here filing me hoping to catch me missing some words. What kind of a doctor roots for somebody who was sick, stay sick?’'

In [60]:
print("Creating Vader sentiment analyzer object")

sentiment = SentimentIntensityAnalyzer()

Creating Vader sentiment analyzer object


In [68]:
# TEST CODE
ls = ["what", "terrible", "idea"]
sentiment.polarity_scores("What a terrible idea!")

{'neg': 0.531, 'neu': 0.469, 'pos': 0.0, 'compound': -0.5255}

In [70]:
# TEST CODE
sentiment.polarity_scores(ls)

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [83]:
def get_tweets_for_user(username, start_date, end_date, num_tweets):
    search_query = username + " -filter:retweets"
    tweets = api.search_tweets(q = search_query, count = num_tweets,
                               tweet_mode="extended",
                           lang="en", result_type="popular", since_id =
                           start_date, until = end_date)
    return tweets

In [88]:
print("Fetching tweets for two candidates")

democratic_candidate_query = "@JohnFetterman OR John Fetterman OR Fetterman"

republic_candidate_query = "@DrOz OR Dr. Oz OR Dr Oz OR Dr. Mehmet Oz OR Mehmet Oz"

Fetching tweets for two candidates


In [ ]:
print("Converting tweets to json")

json_tweets = []

for tweet in tweets:
    json_tweet_str = json.dumps(tweet._json)
    json_tweet = json.loads(json_tweet_str)
    json_tweets.append(json_tweet)

In [ ]:
print("Writing tweets to the file")

TWEETS_DIR = "tweets/"

tweets_file = open(TWEETS_DIR + "tweets.json", "a")

tweets_file_str = json.dumps(json_tweets)

tweets_file.write(tweets_file_str)

In [ ]:
# plot sentiment in a timeseries graph